This notebook outlines the data ingestion/storage process into the S3 bucket.




In [ ]:
!pip install huggingface_hub
!pip install cohere
!pip install transformers
!pip install sumy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 50.6 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21691 sha256=36d402c4ee50fc03ffcd0c6a1a845e6169673cfb8a176b0b465c6a6146f1d2e4
  Stored in directory: /root/.cache/pip/wheels/4d/57/58/7e3d7fedf51fe248b7fcee3df6945ae28638e22cddf01eb92b
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=a46fe3e67a92af713ec4b5574fe7c1b91c603ea92ebd960efb66b6e20ffd308c
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built breadability docopt


# Loading Hugging Face Data

In [2]:
from huggingface_hub import hf_hub_download
import pandas as pd

# huggingface repo id
repo_id = "abisee/cnn_dailymail"


filenames = [
    "3.0.0/test-00000-of-00001.parquet",
    "3.0.0/train-00000-of-00003.parquet",
    "3.0.0/train-00001-of-00003.parquet",
    "3.0.0/train-00002-of-00003.parquet",
    "3.0.0/validation-00000-of-00001.parquet"
]

dfs = {
    filename: pd.read_parquet(hf_hub_download(
        repo_id=repo_id, filename=filename, repo_type="dataset"
    ))
    for filename in filenames
}


test_df = dfs["3.0.0/test-00000-of-00001.parquet"]
train_df = pd.concat([dfs[f] for f in filenames if "train" in f], ignore_index=True)
val_df = dfs["3.0.0/validation-00000-of-00001.parquet"]

##Storing to S3

In [ ]:
#save file to parquet
train_df.to_parquet('train.parquet')

In [ ]:
import boto3
def upload_to_s3(file_path, bucket_name, s3_key):
  s3 = boto3.client('s3')
  try:
    s3.upload_file(file_path, bucket_name, s3_key)
    print(f"File '{file_path}' successfully uploaded to 's3://{bucket_name}/{s3_key}'")
  except Exception as e:
    print(f"Error uploading file: {e}")